Process:

1. Found Twitter accounts tied to 85 airlines worldwide
1. Bookmarked their pages and scraped Twitter handles from bookmarks text file

In [1]:
with open("bookmarks_twitter_airlines.txt", "r") as myfile:
    bookmarks = myfile.read().splitlines()

In [2]:
twitter_airline_handles = [bkmk.split("(")[1].split(")")[0] for bkmk in bookmarks]

In [3]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAElqjwEAAAAA8PRQduIf5nDumMCOqlgqpkL9QHo%3D7d4vdi0UEJNMs9SmNH8nEQf9acD08THiUvzkJq3VfQzQJ4SSEf"

In [4]:
import json
with open('airline_account_handles_and_ids.json') as f:
    airline_handles_and_ids = json.load(f)

In [5]:
import requests
import os
import json
import time
from time import sleep

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(user_id):
    # Replace with user ID below
    return "https://api.twitter.com/2/users/{}/tweets".format(user_id)

def get_params(max_results, next_token=None):
    place_fields = [
        'full_name', 'id', 'country', 'country_code', 
        'name', 'place_type'
    ]
    
    tweet_fields = [
        'id', 'text', 'author_id', 'context_annotations',
        'conversation_id', 'created_at', 'entities', 
        'in_reply_to_user_id', 'lang', 'public_metrics', 
        'source', 'withheld'
    ]
    
    # user_fields = [
    #     'verified'
    # ]
    
    expansions = ['geo.place_id', 'entities.mentions.username']
    
    params = {
        "max_results": max_results,
        #'start_time': start_time,
        #'end_time': end_time,
        'exclude': 'replies',
        'expansions': ",".join(expansions),
        'tweet.fields': ",".join(tweet_fields),
        # 'user.fields': ",".join(user_fields),
        'place.fields': ",".join(place_fields)
    }
    
    # params = {"tweet.fields": "id,text,author_id,context_annotations,conversation_id,created_at", 
    #         "max_results": 5}
    
    # params = {"tweet.fields": ",".join(tweet_fields), 
    #           #'start_time': start_time,
    #           #'end_time': end_time,
    #         "max_results": 5}
    
    # if until_id != "None":
    #     params["until_id"] = until_id
    
    if next_token:
        params["pagination_token"] = next_token
        
    return params


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


# def main():
#     url = create_url()
#     params = get_params()
#     json_response = connect_to_endpoint(url, params)
#     print(json.dumps(json_response, indent=4, sort_keys=True))


# if __name__ == "__main__":
#     main()

In [7]:
save_dir = "airline_tweets_all_NO_REPLIES/"

start_idx = 7
end_idx = 100
for num, airline_info in enumerate(airline_handles_and_ids['data'][start_idx: end_idx]):
    
    print(start_idx + num)
    
    user_id = airline_info['id']
    user_handle = airline_info['username']
    
    print(user_handle)
    print(user_id)

    if not os.path.isdir(save_dir + user_handle):
        os.mkdir(save_dir + user_handle)

    url = create_url(user_id)

    params = get_params(max_results=100)
    json_response = connect_to_endpoint(url, params)
    sleep(5)

    path_to_save_json = save_dir + user_handle + "/" + user_handle + "_" + json_response['meta'].get('newest_id', 'last') + ".json"
    with open(path_to_save_json, 'w') as outfile:
        json.dump(json_response, outfile)
        
    print(f"\tSaved {json_response['meta'].get('result_count')} tweets to " + path_to_save_json + "...")


    next_token = json_response['meta'].get('next_token')


    while next_token:
        params = get_params(max_results=100, next_token=next_token)
        json_response = connect_to_endpoint(url, params)
        sleep(5)


        path_to_save_json = save_dir + user_handle + "/" + user_handle + "_" + json_response['meta'].get('newest_id', 'last') + ".json"
        with open(path_to_save_json, 'w') as outfile:
            json.dump(json_response, outfile)

            
        print(f"\tSaved {json_response['meta'].get('result_count')} tweets to " + path_to_save_json + "...")
        

        next_token = json_response['meta'].get('next_token')
        
print("Done.")

7
flyvolaris
227100749
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1599146470246318080.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1571627636329418752.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1543595425751453696.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1516491832623976448.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1491131258943860738.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1464111697422020612.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1434890969002954753.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1412833799075815424.json...
200
	Saved 50 tweets to airline_tweets_all_NO_REPLIES/flyvolaris/flyvolaris_1385986958913011713.json...
8
Iberia_en
142608485
200
	Saved 

In [9]:
json_response

{'meta': {'result_count': 0,
  'previous_token': '10iy0xk016rrqp9x9wnljt1so5tsxc33d8iyfilllzqt'}}

In [22]:
airline_info

{'id': '260187407', 'name': 'Middle East Airlines', 'username': 'MEAAIRLIBAN'}

In [13]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "2244994945",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to softwares, apps, communication equipments, hardwares",
                        "id": "1557697333571112960",
                        "name": "Technology Business"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
              

In [8]:
# See how many tweets you have
import os

tweet_count = 0
save_dir = "airline_tweets_all_NO_REPLIES/"
for airline_filename in os.listdir(save_dir):
    for airline_tweet_json_filename in os.listdir(save_dir + airline_filename):
        try:
            with open(f'{save_dir}{airline_filename}/{airline_tweet_json_filename}') as f:
                data = json.load(f)
            tweet_count += data['meta']['result_count']
            print(tweet_count)
        except IsADirectoryError:
            continue

100
150
250
350
450
550
650
750
850
950
1000
1100
1199
1298
1398
1498
1598
1698
1798
1897
1996
2095
2195
2295
2395
2445
2545
2645
2745
2845
2945
3045
3145
3245
3345
3395
3495
3595
3645
3745
3845
3945
4045
4145
4245
4345
4445
4545
4636
4735
4835
4835
4935
5035
5135
5185
5285
5385
5485
5585
5685
5785
5885
5984
6084
6183
6282
6382
6479
6579
6579
6675
6775
6875
6975
7075
7102
7202
7302
7402
7502
7602
7700
7800
7900
7997
8097
8105
8205
8305
8405
8505
8605
8705
8805
8905
9005
9016
9116
9216
9316
9416
9466
9566
9666
9766
9866
9966
10066
10166
10266
10268
10368
10468
10568
10668
10768
10868
10968
11068
11168
11268
11367
11467
11564
11576
11676
11776
11876
11976
12076
12176
12276
12276
12376
12474
12574
12674
12774
12874
12974
12980
13080
13180
13280
13380
13480
13530
13630
13730
13830
13930
14030
14130
14230
14330
14430
14530
14630
14679
14779
14879
14979
15079
15179
15279
15328
15428
15528
15628
15657
15757
15857
15957
16057
16157
16257
16357
16456
16555
16555
16652
16750
16849
16948
17048
17